# Library

In [1]:
import pandas as pd
import pickle
import pandas as pd
pd.options.plotting.backend = "plotly"
import util
from sklearn.metrics import mean_squared_error
import pickle
import autokeras as ak
from tensorflow.keras.models import load_model

# Load data and set index

In [2]:
dataset = pd.read_csv('../data_train\df_raw.csv')
dataset.index = dataset.time
dataset =dataset.drop(['time'],axis=1)
dataset

,temperature,co2,iaq,deviceId
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,E7184ABA7A055FD1
2022-11-07 23:15:25+11:00,23.957020,1065.705811,179.0,7BB92D02D696C5C5
2022-11-07 23:15:55+11:00,24.142492,651.757324,79.0,99DD33EFB7990A71
2022-11-07 23:15:57+11:00,23.676592,1170.204346,198.0,1DA36D3BE34D559A
2022-11-07 23:16:07+11:00,24.771078,1066.842407,179.0,B84C4503F361D64A
...,...,...,...,...
2022-11-11 13:59:16+11:00,23.982397,801.501953,119.0,7BB92D02D696C5C5
2022-11-11 13:59:39+11:00,24.150066,617.686890,60.0,99DD33EFB7990A71
2022-11-11 13:59:47+11:00,24.403280,713.524902,98.0,1DA36D3BE34D559A


# Seperate data by sensors id

In [3]:
dfs = util.df_group_by_col(df=dataset, col='deviceId')
dataset = dfs[0]
dataset

,temperature,co2,iaq,deviceId
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,E7184ABA7A055FD1
2022-11-07 23:16:18+11:00,24.782167,1056.472656,177.0,E7184ABA7A055FD1
2022-11-07 23:17:18+11:00,24.789717,1065.587769,179.0,E7184ABA7A055FD1
2022-11-07 23:18:18+11:00,24.784685,1064.388428,179.0,E7184ABA7A055FD1
2022-11-07 23:19:18+11:00,24.792234,1063.462036,178.0,E7184ABA7A055FD1
...,...,...,...,...
2022-11-11 13:56:08+11:00,24.724291,646.477234,76.0,E7184ABA7A055FD1
2022-11-11 13:57:08+11:00,24.704157,645.051270,75.0,E7184ABA7A055FD1
2022-11-11 13:58:08+11:00,24.699125,653.881470,80.0,E7184ABA7A055FD1


# Setup Targets

In [4]:
# co2 as target
dataset['target'] = dataset['co2'].shift(-15)
dataset = dataset[:-15]
dataset_numeric= dataset.drop(['deviceId'], axis=1)
dataset_numeric

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_9184\2727454598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['target'] = dataset['co2'].shift(-15)


,temperature,co2,iaq,target
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,1042.386719
2022-11-07 23:16:18+11:00,24.782167,1056.472656,177.0,1056.373047
2022-11-07 23:17:18+11:00,24.789717,1065.587769,179.0,1062.436279
2022-11-07 23:18:18+11:00,24.784685,1064.388428,179.0,1049.018311
2022-11-07 23:19:18+11:00,24.792234,1063.462036,178.0,1053.749268
...,...,...,...,...
2022-11-11 13:41:08+11:00,24.736872,613.454834,57.0,646.477234
2022-11-11 13:42:08+11:00,24.754488,615.277893,58.0,645.051270
2022-11-11 13:43:08+11:00,24.714224,636.333313,70.0,653.881470


# Data scaling - Normalization

In [5]:
dataset_scaled =(dataset_numeric-dataset_numeric.min())/(dataset_numeric.max()-dataset_numeric.min())
dataset_scaled

,temperature,co2,iaq,target
time,,,,
2022-11-07 23:15:18+11:00,0.522236,0.613637,0.842105,0.585213
2022-11-07 23:16:18+11:00,0.519366,0.603967,0.828947,0.603834
2022-11-07 23:17:18+11:00,0.523670,0.616103,0.842105,0.611907
2022-11-07 23:18:18+11:00,0.520801,0.614506,0.842105,0.594042
2022-11-07 23:19:18+11:00,0.525105,0.613273,0.835526,0.600341
...,...,...,...,...
2022-11-11 13:41:08+11:00,0.493541,0.014121,0.039474,0.058088
2022-11-11 13:42:08+11:00,0.503585,0.016548,0.046053,0.056189
2022-11-11 13:43:08+11:00,0.480629,0.044582,0.125000,0.067946


## De-normalization factors

In [6]:
max_d = dataset_numeric['target'].max()
min_d = dataset_numeric['target'].min()
#denormalized_d = normalized_d * (max_d - min_d) + min_d
print(max_d, min_d)

1353.922607421875 602.8489379882812


# Create lookback feature

In [7]:
lookback = 60

features = list(dataset_scaled.drop(['target'],axis=1).columns)
dataset_lb = dataset_scaled[:]

for i in range(1,lookback):    
    features_lb = list(map(lambda ls: ls+"_"+str(i), features))
    dataset_lb[features_lb]= dataset_scaled.drop(['target'],axis=1).shift(i)

dataset_lb = dataset_lb[lookback:]
dataset_lb
dataset = dataset_lb
dataset

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_9184\223088852.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_lb[features_lb]= dataset_scaled.drop(['target'],axis=1).shift(i)
C:\Users\ANPCC\AppData\Local\Temp\ipykernel_9184\223088852.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_lb[features_lb]= dataset_scaled.drop(['target'],axis=1).shift(i)
C:\Users\ANPCC\AppData\Local\Temp\ipykernel_9184\223088852.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,temperature,co2,iaq,target,temperature_1,co2_1,iaq_1,temperature_2,co2_2,iaq_2,...,iaq_56,temperature_57,co2_57,iaq_57,temperature_58,co2_58,iaq_58,temperature_59,co2_59,iaq_59
time,,,,,,,,,,,,,,,,,,,,,
2022-11-08 00:15:18+11:00,0.487803,0.589232,0.809211,0.551340,0.487803,0.588067,0.809211,0.484933,0.588588,0.809211,...,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105,0.519366,0.603967,0.828947
2022-11-08 00:16:18+11:00,0.479212,0.582024,0.802632,0.582618,0.487803,0.589232,0.809211,0.487803,0.588067,0.809211,...,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105
2022-11-08 00:17:18+11:00,0.479194,0.587984,0.809211,0.581631,0.479212,0.582024,0.802632,0.487803,0.589232,0.809211,...,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105
2022-11-08 00:18:18+11:00,0.477759,0.582083,0.802632,0.588554,0.479194,0.587984,0.809211,0.479212,0.582024,0.802632,...,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526
2022-11-08 00:19:18+11:00,0.479212,0.582584,0.802632,0.578412,0.477759,0.582083,0.802632,0.479194,0.587984,0.809211,...,0.835526,0.510757,0.610349,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11 13:41:08+11:00,0.493541,0.014121,0.039474,0.058088,0.490671,0.013907,0.039474,0.476325,0.013700,0.039474,...,0.092105,0.407459,0.032102,0.092105,0.406023,0.034517,0.098684,0.390241,0.033114,0.092105
2022-11-11 13:42:08+11:00,0.503585,0.016548,0.046053,0.056189,0.493541,0.014121,0.039474,0.490671,0.013907,0.039474,...,0.078947,0.408892,0.031365,0.092105,0.407459,0.032102,0.092105,0.406023,0.034517,0.098684
2022-11-11 13:43:08+11:00,0.480629,0.044582,0.125000,0.067946,0.503585,0.016548,0.046053,0.493541,0.014121,0.039474,...,0.085526,0.408892,0.026473,0.078947,0.408892,0.031365,0.092105,0.407459,0.032102,0.092105


# Train valid test split

In [8]:
val_split = int(len(dataset) * 0.7)

data_train = dataset[:val_split]
data_validation = dataset[val_split:]

test_split = int(len(data_validation) * 0.5)

data_test = data_validation[test_split:]
data_validation = data_validation[:test_split]

#===============================================================

print(data_train.shape, data_validation.shape ,data_test.shape )
data_train.head()

(3309, 181) (709, 181) (710, 181)


,temperature,co2,iaq,target,temperature_1,co2_1,iaq_1,temperature_2,co2_2,iaq_2,...,iaq_56,temperature_57,co2_57,iaq_57,temperature_58,co2_58,iaq_58,temperature_59,co2_59,iaq_59
time,,,,,,,,,,,,,,,,,,,,,
2022-11-08 00:15:18+11:00,0.487803,0.589232,0.809211,0.551340,0.487803,0.588067,0.809211,0.484933,0.588588,0.809211,...,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105,0.519366,0.603967,0.828947
2022-11-08 00:16:18+11:00,0.479212,0.582024,0.802632,0.582618,0.487803,0.589232,0.809211,0.487803,0.588067,0.809211,...,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105
2022-11-08 00:17:18+11:00,0.479194,0.587984,0.809211,0.581631,0.479212,0.582024,0.802632,0.487803,0.589232,0.809211,...,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105
2022-11-08 00:18:18+11:00,0.477759,0.582083,0.802632,0.588554,0.479194,0.587984,0.809211,0.479212,0.582024,0.802632,...,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526
2022-11-08 00:19:18+11:00,0.479212,0.582584,0.802632,0.578412,0.477759,0.582083,0.802632,0.479194,0.587984,0.809211,...,0.835526,0.510757,0.610349,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526


# Final feature selection

In [9]:
X_train = data_train.drop(['target'],axis=1).astype("float64")
X_val = data_validation.drop(['target'],axis=1).astype("float64")
X_test = data_test.drop(['target'],axis=1).astype("float64")

y_train = data_train[["target"]].astype("float64")
y_val = data_validation[["target"]].astype("float64")
y_test = data_test[["target"]].astype("float64")

num_feature = X_train.shape[1]
    
print(X_train.shape, X_val.shape,X_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)
print(num_feature)


(3309, 180) (709, 180) (710, 180)
(3309, 1) (709, 1) (710, 1)
180


# Define model

In [10]:
# Initialize the structured data regressor.
model = ak.StructuredDataRegressor(overwrite=True, max_trials=3)  # It tries 3 different models.

# Training

In [11]:
# Feed the structured data regressor with training data.
model.fit(X_train, y_train, epochs=50)

Trial 3 Complete [00h 00m 44s]
val_loss: 0.012798507697880268

Best val_loss So Far: 0.011509585194289684
Total elapsed time: 00h 02m 20s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
104/104 [==============================] - 3s 4ms/step - loss: 0.2706 - mean_squared_error: 0.2706
Epoch 2/50
104/104 [==============================] - 0s 4ms/step - loss: 0.1519 - mean_squared_error: 0.1519
Epoch 3/50
104/104 [==============================] - 0s 4ms/step - loss: 0.0938 - mean_squared_error: 0.0938
Epoch 4/50
104/104 [==============================] - 0s 4ms/step - loss: 0.0692 - mean_squared_error: 0.0692
Epoch 5/50
104/104 [==============================] - 0s 5ms/step - loss: 0.0721 - mean_squared_error: 0.0721
Epoch 6/50
104/104 [==============================] - 0s 4ms/step - loss: 0.0570 - mean_squared_error: 0.0570
Epoch 7/50
104/104 [==============================] - 0s 5ms/step - loss: 0.0611 - mean_squared_error: 0.0611
Epoch 8/50
104/104 [==============================] - 

# Inference

In [12]:
# Predict with the best model(includes original training data).
y_pred = model.predict(X_test) 
y_pred

23/23 [==============================] - 0s 4ms/step


array([[0.22426566],
       [0.19245343],
       [0.2386412 ],
       [0.21594194],
       [0.18764721],
       [0.21821173],
       [0.22099382],
       [0.14840382],
       [0.18783583],
       [0.19678128],
       [0.18250506],
       [0.19680324],
       [0.2381193 ],
       [0.1797381 ],
       [0.23727341],
       [0.19775903],
       [0.20319001],
       [0.23507531],
       [0.21234077],
       [0.21498951],
       [0.22582887],
       [0.18384506],
       [0.24264458],
       [0.2682889 ],
       [0.26625755],
       [0.18592861],
       [0.2690841 ],
       [0.21943447],
       [0.19769742],
       [0.21131524],
       [0.22535543],
       [0.19341955],
       [0.22841969],
       [0.21700132],
       [0.18440065],
       [0.21924517],
       [0.25005716],
       [0.2626323 ],
       [0.24796392],
       [0.21696223],
       [0.2049147 ],
       [0.26274535],
       [0.22532941],
       [0.21851185],
       [0.27040923],
       [0.24345064],
       [0.27084038],
       [0.264

# Evaluation

In [13]:
pred_val =model.predict(X_val) * (max_d - min_d)+ min_d
true_val =y_val * (max_d - min_d) + min_d

pred_test =model.predict(X_test) * (max_d - min_d)+ min_d
true_test = y_test * (max_d - min_d) + min_d

pred_train =model.predict(X_train) * (max_d - min_d)+ min_d
true_train = y_train * (max_d - min_d) + min_d

print(mean_squared_error(model.predict(X_val), y_val))
print(mean_squared_error(model.predict(X_test), y_test))

print(mean_squared_error(pred_train , true_train))
print(mean_squared_error(pred_val , true_val))
print(mean_squared_error(pred_test , true_test))

23/23 [==============================] - 0s 4ms/step
0.016253543388168173
23/23 [==============================] - 0s 4ms/step
0.018540793280967315
4792.178378148424
9168.813322450545
10459.077895892091


# Visualize Result

In [14]:
df_viz_train = pd.DataFrame()
df_viz_train['pred_train'] =pred_train.reshape(-1)
df_viz_train['true_train'] =true_train.values
df_viz_train.plot()

In [15]:
df_viz_val = pd.DataFrame()
df_viz_val['pred_val'] =pred_val.reshape(-1)
df_viz_val['true_val'] =true_val.values
df_viz_val.plot()

In [16]:
df_viz_test = pd.DataFrame()
df_viz_test['pred_test'] =pred_test.reshape(-1)
df_viz_test['true_test'] =true_test.values
df_viz_test.plot()

# Save model

In [17]:
file_name = "../saved_model/autokeras_reg.pkl"
# save
pickle.dump(model, open(file_name, "wb"))

# Load model

In [18]:
# load
model_loaded = pickle.load(open(file_name, "rb"))

# Load model Inference

In [19]:
result = model_loaded.predict(X_test)
result_denor = result  * (max_d - min_d) + min_d
result_denor

23/23 [==============================] - 0s 8ms/step


array([[ 771.28894],
       [ 747.3956 ],
       [ 782.08606],
       [ 765.03723],
       [ 743.7858 ],
       [ 766.742  ],
       [ 768.83154],
       [ 714.31116],
       [ 743.9275 ],
       [ 750.6462 ],
       [ 739.9237 ],
       [ 750.66266],
       [ 781.6941 ],
       [ 737.84546],
       [ 781.0587 ],
       [ 751.38055],
       [ 755.4596 ],
       [ 779.40784],
       [ 762.3325 ],
       [ 764.3219 ],
       [ 772.4631 ],
       [ 740.9301 ],
       [ 785.0929 ],
       [ 804.35364],
       [ 802.828  ],
       [ 742.495  ],
       [ 804.9509 ],
       [ 767.6604 ],
       [ 751.3342 ],
       [ 761.56226],
       [ 772.1075 ],
       [ 748.1213 ],
       [ 774.40894],
       [ 765.8329 ],
       [ 741.3474 ],
       [ 767.5182 ],
       [ 790.6603 ],
       [ 800.10516],
       [ 789.08813],
       [ 765.8036 ],
       [ 756.755  ],
       [ 800.19006],
       [ 772.0879 ],
       [ 766.9674 ],
       [ 805.94617],
       [ 785.6983 ],
       [ 806.27   ],
       [ 801.